# Stock data analysis

In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
openai = OpenAI()

In [4]:
from urllib.parse import urlencode
import datetime

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [5]:
class YahooFinanceWebsite:
    def __init__(self, stock_symbol):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.stock_symbol = stock_symbol.upper()

    def __build_url(self, params):
        base_url = f"https://finance.yahoo.com/quote/{self.stock_symbol}/history/"
        query_string = urlencode(params)
        return f"{base_url}?{query_string}"

    def get_stock_data(self):
        datetime_now = datetime.datetime.now()
        datetime_year_ago = datetime_now - datetime.timedelta(days=365)
        params = {"frequency": "1wk", "period1": datetime_year_ago.timestamp(), "period2": datetime_now.timestamp()}
        url = self.__build_url(params)
        response = requests.get(url, headers=headers)

        soup = BeautifulSoup(response.content, 'html.parser')
        
        title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        html_table_data = soup.find("table")

        return title, html_table_data

In [6]:
def build_stock_analysis_prompt(stock_symbol, title, stock_table_data):
    sys_prompt = r"""You are an assistant that analyzes the contents of HTML formated table that contains data on a specific stock.
    The HTML table contains the date, open price, close price, low and highs aggregated for every week over one year timeframe.
    Ignoring text, tags or html attributes that might be navigation related. 
    Respond in Markdown format"""
    
    user_prompt = f"The data provided below in the HTML table format for {stock_symbol} from the Yahoo Finances.\
    Make the explaination easy enough for a newbie to understand. \
    Analyze and Summarize the trends on this stock:\n{stock_table_data}\n\n\
    Also, calculate the total returns in percentage one could have expected over this period."
    
    return [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt}
    ]

In [7]:
def analyze_stock_trends(stock_symbol):
    stock_data_page = YahooFinanceWebsite(stock_symbol)
    title, stock_table_data = stock_data_page.get_stock_data()
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = build_stock_analysis_prompt(stock_symbol, title, stock_table_data)
    )
    return response.choices[0].message.content

def display_analysis(stock_symbol):
    display(Markdown(analyze_stock_trends(stock_symbol)))

In [8]:
display_analysis("GOOG")

## Analysis of GOOG Stock Trends

The provided data consists of weekly stock price movements of Google (ticker symbol: GOOG) over a year, with relevant details such as dates, opening prices, high prices, low prices, closing prices, adjusted closing prices, and trading volumes.

### Summary of Trends

1. **General Price Movement**:
   - The stock opened at **$156.64** on April 17, 2025, and closed at **$153.36** by the same date.
   - The highest price observed in the data was **$208.70** on February 3, 2025, while the lowest was **$142.66** on April 7, 2025.

2. **Price Fluctuations**:
   - There have been notable price fluctuations throughout the period:
     - A significant drop occurred from late January 2025 (around **$205.60**) to March 2025, where it dropped to **$147.74**.
     - The price rose back up to around **$170** by the end of March.
     - Recently, it dropped again, indicating possible volatility.

3. **Volume Analysis**:
   - Trading volumes have varied significantly. For example:
     - The highest volume noted was **187,459,400 shares** on April 7, 2025, coinciding with a considerable price rise.
     - Lower trading volumes were observed during price corrections, indicating that fewer investors were trading during those downturn periods.

4. **Return on Investment**:
   - To calculate the total returns from the start to the end of this period, we consider:
     - **Closing Price on Start Date (April 17, 2025)**: **$153.36**
     - **Closing Price on End Date (April 14, 2025)**: **$153.36**
   - The formula to calculate return is:
     \[
     \text{Total Return (\%)} = \left( \frac{\text{End Price} - \text{Start Price}}{\text{Start Price}} \right) \times 100
     \]
   - Plugging in values:
     \[
     \text{Total Return (\%)} = \left( \frac{153.36 - 156.64}{156.64} \right) \times 100 \approx -2.85\%
     \]

### Conclusion

- Overall, **GOOG stock has seen both ups and downs**, with fluctuations indicating market volatility. 
- If you had invested in GOOG at the beginning of this timeframe, you would have faced **a decrease of approximately 2.85%** in your investment by the date analyzed. This slight loss reflects the stock's tendency to experience volatility and highlights the risks involved with stock investing. 

New investors should consider both the potential for losses alongside the chances for gains when evaluating their investment in GOOG stock.

In [ ]:
display_analysis("PFE")

In [9]:
display_analysis("AAPL")

# AAPL Stock Data Analysis (HTML Table)

## Overview

The table provided contains weekly stock data for Apple Inc. (AAPL) from the last year. The key columns we will focus on are:
- **Open**: The stock price at the beginning of the week
- **Close**: The stock price at the end of the week
- **High**: The highest price during the week
- **Low**: The lowest price during the week
- **Volume**: The number of shares traded.

## Trends and Summary

### Price Trend
1. **Volatility**: The stock prices show considerable fluctuations over the observed period, with both highs and lows varying significantly.
2. **Peak and Trough**:
   - The highest recorded opening price was on **Apr 14, 2025**, at **211.44**.
   - The lowest point was around **169.30** on **Apr 22, 2024**.
3. **Closing Trend**:
   - The closing price experienced peaks and troughs throughout the year, notably jumping from **182.19** (Apr 6, 2024) to **196.98** (Apr 14, 2025).
   - There is a visible upward trend in the closing prices towards the latest dates, indicating rising investor confidence and potential strong financial fundamentals.

### Volume Trends
- The trading volume varied significantly, with some days reflecting heavy trading (over **600M** shares) suggesting high investor interest during those weeks.
- High volume days often correlate with large price movements, indicating reactions to market news or company announcements.

### Returns Calculation
To calculate the total returns:
- Use the closing price from **April 15, 2024** (the beginning of the observed period) and the latest closing price from **April 17, 2025**.

1. **Closing Price on Apr 15, 2024**: **165.00**
2. **Closing Price on Apr 17, 2025**: **196.98**

**Formula for Total Returns**:
\[ \text{Total Returns} (\%) = \left( \frac{\text{Final Price} - \text{Initial Price}}{\text{Initial Price}} \right) \times 100 \]

Calculating this:
\[ \text{Total Returns} (\%) = \left( \frac{196.98 - 165.00}{165.00} \right) \times 100 \approx 19.37\% \]

## Conclusion
- Over the past year, AAPL stock has seen significant fluctuations, suggesting investment opportunities during both peaks and troughs.
- A total return of **approximately 19.37%** reflects an overall positive performance for investors holding the stock during this period, despite the short-term volatility.